In [104]:
"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Google CLI Auth"""
# !gcloud auth application-default login


import google.auth
credentials, project_id = google.auth.default()

#@markdown Please fill in the value below with your GCP project ID and then run the cell.
# PROJECT_ID = "talk2data-genai-sa" #@param {type:"string"}
PROJECT_ID = input("Please enter the Google Cloud Project ID ")

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
PROJECT_ID
# !gcloud auth application-default set-quota-project {PROJECT_ID}

import os
os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

Updated property [core/project].


In [105]:
import os
import sys
import configparser
config = configparser.ConfigParser()
import pandas as pd
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector
from google.cloud import bigquery

module_prefixes = ["Talk2Data", "open-data-qna", "applied-ai-engineering-samples"]

current_dir = os.getcwd()
while current_dir != os.path.dirname(current_dir):  # Loop until root dir
    if any(prefix in current_dir for prefix in module_prefixes):
        config_path = os.path.join(current_dir, 'config.ini')
        if os.path.exists(config_path):
            config.read(config_path)
            root_dir = current_dir
            break  # Stop searching once found
    current_dir = os.path.dirname(current_dir)  # Move to parent dir

if not 'root_dir' in locals():  # If not found in any parent dir
    raise FileNotFoundError("config.ini not found in current or parent directories.")

import configparser
config = configparser.ConfigParser()
config.read(root_dir+'/config.ini')

PROJECT_ID = config['GCP']['PROJECT_ID']
DATA_SOURCE = config['CONFIG']['DATA_SOURCE']
VECTOR_STORE = config['CONFIG']['VECTOR_STORE']
PG_SCHEMA = config['PGCLOUDSQL']['PG_SCHEMA']
PG_DATABASE = config['PGCLOUDSQL']['PG_DATABASE']
PG_USER = config['PGCLOUDSQL']['PG_USER']
PG_REGION = config['PGCLOUDSQL']['PG_REGION'] 
PG_INSTANCE = config['PGCLOUDSQL']['PG_INSTANCE'] 
PG_PASSWORD = config['PGCLOUDSQL']['PG_PASSWORD']
BQ_OPENDATAQNA_DATASET_NAME = config['BIGQUERY']['BQ_OPENDATAQNA_DATASET_NAME']
BQ_LOG_TABLE_NAME = config['BIGQUERY']['BQ_LOG_TABLE_NAME'] 
BQ_DATASET_REGION = config['BIGQUERY']['BQ_DATASET_REGION']
BQ_DATASET_NAME = config['BIGQUERY']['BQ_DATASET_NAME']
BQ_TABLE_LIST = config['BIGQUERY']['BQ_TABLE_LIST']

In [106]:
current_dir = os.getcwd()
root_dir = os.path.expanduser('~')  # Start at the user's home directory

while current_dir != root_dir:
    for dirpath, dirnames, filenames in os.walk(current_dir):
        config_path = os.path.join(dirpath, 'known_good_sql.csv')
        if os.path.exists(config_path):
            file_path = config_path  # Update root_dir to the found directory
            break  # Stop outer loop once found

    current_dir = os.path.dirname(current_dir)

print("Known Good SQL Found at Path :: "+file_path)

# Load the file
df_kgq = pd.read_csv(file_path)
df_kgq = df_kgq.loc[:, ["prompt", "sql", "database_name"]]
df_kgq = df_kgq.dropna()

print('Known Good SQLs Loaded into a Dataframe')

Known Good SQL Found at Path :: /home/mokshazna/Talk2Data/scripts/known_good_sql.csv


##Specify mode for loading the known good sql

The known good sql can loaded in two modes:
* Append mode: Apended to the existing KGQ in the vector store 
* Refresh mode: Delete the existing KGQ and create of fresh copy from KGQ in known_good_sql.csv file

In [2]:
loading_mode = 'append' # Options 'append' or 'referesh'

assert loading_mode in {'append', 'referesh'}, "⚠️ Invalid loading_mode. Must be 'append' and 'referesh'"

In [108]:
# If you have Known Good Queries, load them to known_good_sql.csv file; 
# These will be used as few shot examples for query generation. 
if loading_mode == 'refresh':

    from embeddings.kgq_embeddings import setup_kgq_table
    # Delete any old tables and create a new table to KGQ embeddings
    if VECTOR_STORE=='bigquery-vector':
        await(setup_kgq_table(project_id=PROJECT_ID,
                            instance_name=None,
                            database_name=None,
                            schema=BQ_OPENDATAQNA_DATASET_NAME,
                            database_user=None,
                            database_password=None,
                            region=BQ_DATASET_REGION,
                            VECTOR_STORE = VECTOR_STORE
                            ))

    elif VECTOR_STORE=='cloudsql-pgvector':
        await(setup_kgq_table(project_id=PROJECT_ID,
                            instance_name=PG_INSTANCE,
                            database_name=PG_DATABASE,
                            schema=None,
                            database_user=PG_USER,
                            database_password=PG_PASSWORD,
                            region=PG_REGION,
                            VECTOR_STORE = VECTOR_STORE
                            ))


print("Adding Known Good Queries to the Vector store.....")

import os
import pandas as pd
current_dir = os.getcwd()
root_dir = os.path.expanduser('~')  # Start at the user's home directory

while current_dir != root_dir:
    for dirpath, dirnames, filenames in os.walk(current_dir):
        config_path = os.path.join(dirpath, 'known_good_sql.csv')
        if os.path.exists(config_path):
            file_path = config_path  # Update root_dir to the found directory
            break  # Stop outer loop once found

    current_dir = os.path.dirname(current_dir)

print("Known Good SQL Found at Path :: "+file_path)

# Load the file
df_kgq = pd.read_csv(file_path)
df_kgq = df_kgq.loc[:, ["prompt", "sql", "database_name"]]
df_kgq = df_kgq.dropna()

from embeddings.kgq_embeddings import store_kgq_embeddings
# Add KGQ to the vector store
if VECTOR_STORE=='bigquery-vector':
    await(store_kgq_embeddings(df_kgq,
                                project_id=PROJECT_ID,
                                instance_name=None,
                                database_name=BQ_OPENDATAQNA_DATASET_NAME,
                                schema=BQ_DATASET_NAME,
                                database_user=None,
                                database_password=None,
                                region=BQ_DATASET_REGION,
                                VECTOR_STORE = VECTOR_STORE
                                ))

elif VECTOR_STORE=='cloudsql-pgvector':
    await(store_kgq_embeddings(df_kgq,
                                project_id=PROJECT_ID,
                                instance_name=PG_INSTANCE,
                                database_name=PG_DATABASE,
                                schema=None,
                                database_user=PG_USER,
                                database_password=PG_PASSWORD,
                                region=PG_REGION,
                                VECTOR_STORE = VECTOR_STORE
                                ))
print('Done!!')

Good SQL Examples have been loaded to BigQuery Vector
